In [0]:
-- カタログ/スキーマ
CREATE CATALOG IF NOT EXISTS users;
CREATE SCHEMA  IF NOT EXISTS users.yukiteru_koide;
USE CATALOG users;
USE users.yukiteru_koide;

-- 1) 生CSVスキーマそのままのステージ表（取り込み専用）
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_logs_bronze_src (
  id STRING,
  createdDateTime STRING,
  userPrincipalName STRING,
  userId STRING,
  appDisplayName STRING,
  appId STRING,
  ipAddress STRING,
  clientAppUsed STRING,
  isInteractive STRING,
  authenticationRequirement STRING,
  conditionalAccessStatus STRING,
  deviceOperatingSystem STRING,
  browser STRING,
  locationCountryOrRegion STRING,
  locationCity STRING,
  statusErrorCode STRING,
  statusFailureReason STRING,
  statusAdditionalDetails STRING
) USING delta;

-- 2) CSV → ステージ表（※ここは“パスを文字列で”）
COPY INTO users.yukiteru_koide.signin_logs_bronze_src
FROM 'dbfs:/Volumes/users/yukiteru_koide/security/entra_signin_logs_10k.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true');

-- 3) 派生列付きの本番Bronze表（パーティション付き）
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_logs_bronze
USING delta
PARTITIONED BY (p_date) AS
SELECT
  *,
  to_timestamp(createdDateTime)                            AS created_ts,
  CAST(statusErrorCode AS INT)                             AS statusErrorCode_int,
  CASE lower(isInteractive)
    WHEN 'true' THEN true WHEN 'false' THEN false ELSE NULL END AS isInteractive_bool,
  to_date(to_timestamp(createdDateTime))                   AS p_date,
  current_timestamp()                                      AS ingestion_ts,
  '/Volumes/users/yukiteru_koide/security/entra_signin_logs_10k.csv' AS source_path
FROM users.yukiteru_koide.signin_logs_bronze_src;

OPTIMIZE users.yukiteru_koide.signin_logs_bronze;
